In [4]:
import numpy as np
import pandas as pd 
import os, cv2
from tqdm import tqdm
from sklearn.model_selection import StratifiedShuffleSplit

from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.applications import *

In [5]:
x_train = []
y_train = []

label_map = {   "Black-grass"               :0,
                "Charlock"                  :1,
                "Cleavers"                  :2,
                "Common Chickweed"          :3,
                "Common wheat"              :4,
                "Fat Hen"                   :5,
                "Loose Silky-bent"          :6,
                "Maize"                     :7,
                "Scentless Mayweed"         :8,
                "Shepherds Purse"           :9,
                "Small-flowered Cranesbill" :10,
                "Sugar beet"                :11}

dim = 256

# Preparing training data
dirs = os.listdir("/input/plant_seedlings_classification/train/")
for k in tqdm(range(len(dirs))):    # Directory
    files = os.listdir("/input/plant_seedlings_classification/train/{}".format(dirs[k]))
    for f in range(len(files)):     # Files
        img = cv2.imread('/input/plant_seedlings_classification/train/{}/{}'.format(dirs[k], files[f]))
        targets = np.zeros(12)
        targets[label_map[dirs[k]]] = 1 
        x_train.append(cv2.resize(img, (dim, dim)))
        y_train.append(targets)
    
y_train = np.array(y_train, np.uint8)
x_train = np.array(x_train, np.float32)

print(x_train.shape)
print(y_train.shape)

#x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.01, random_state=42)
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.16, random_state=42) # Want a balanced split for all the classes
for train_index, test_index in sss.split(x_train, y_train):
    print("Using {} for training and {} for validation".format(len(train_index), len(test_index)))
    x_train, x_valid = x_train[train_index], x_train[test_index]
    y_train, y_valid = y_train[train_index], y_train[test_index]

100%|██████████| 12/12 [00:56<00:00,  4.97s/it]


(4750, 256, 256, 3)
(4750, 12)


In [7]:
datagen = ImageDataGenerator( horizontal_flip=True, 
                              vertical_flip=True,
                            rescale = 1./255)
                                      
weights = os.path.join('', '/output/weights.h5')

epochs = 100
learning_rate = 0.001
batch_size = 32

callbacks = [ EarlyStopping(monitor='val_loss', patience=5, verbose=0), 
              ModelCheckpoint(weights, monitor='val_loss', save_best_only=True, verbose=0),
              ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)]

base_model = InceptionResNetV2(input_shape=(dim, dim, 3), include_top=False, weights='imagenet', pooling='avg') # Average pooling reduces output dimensions
x = base_model.output
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(12, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=learning_rate), metrics=['accuracy'])

219062272/219055592 [==============================] - 14s 0us/step


In [10]:
# Training whole convnet
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train)/batch_size, 
                    validation_data=datagen.flow(x_valid, y_valid, batch_size=batch_size), 
                    validation_steps=len(x_valid)/batch_size,
                    callbacks=callbacks,
                    epochs=21, 
                    verbose=1)

Epoch 1/21
125/124 [==============================] - 415s 3s/step - loss: 0.9355 - acc: 0.6990 - val_loss: 3.7049 - val_acc: 0.3105
Epoch 2/21
125/124 [==============================] - 295s 2s/step - loss: 0.4749 - acc: 0.8487 - val_loss: 5.0824 - val_acc: 0.4013
Epoch 3/21
125/124 [==============================] - 298s 2s/step - loss: 0.3616 - acc: 0.8774 - val_loss: 1.5777 - val_acc: 0.6974
Epoch 4/21
125/124 [==============================] - 297s 2s/step - loss: 0.3159 - acc: 0.8973 - val_loss: 0.8135 - val_acc: 0.7816
Epoch 5/21
125/124 [==============================] - 297s 2s/step - loss: 0.2571 - acc: 0.9163 - val_loss: 0.2739 - val_acc: 0.9105
Epoch 6/21
125/124 [==============================] - 294s 2s/step - loss: 0.2458 - acc: 0.9199 - val_loss: 2.2538 - val_acc: 0.6684
Epoch 7/21
125/124 [==============================] - 294s 2s/step - loss: 0.2378 - acc: 0.9216 - val_loss: 0.3852 - val_acc: 0.9066
Epoch 8/21
125/124 [==============================] - 296s 2s/step - 

In [11]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train)/batch_size, 
                    validation_data=datagen.flow(x_valid, y_valid, batch_size=batch_size), 
                    validation_steps=len(x_valid)/batch_size,
                    callbacks=callbacks,
                    epochs=21, 
                    verbose=1)

Epoch 1/21
125/124 [==============================] - 294s 2s/step - loss: 0.0204 - acc: 0.9945 - val_loss: 0.0844 - val_acc: 0.9750
Epoch 2/21
125/124 [==============================] - 294s 2s/step - loss: 0.0247 - acc: 0.9922 - val_loss: 0.0746 - val_acc: 0.9763
Epoch 3/21
125/124 [==============================] - 294s 2s/step - loss: 0.0225 - acc: 0.9934 - val_loss: 0.0658 - val_acc: 0.9737
Epoch 4/21
125/124 [==============================] - 294s 2s/step - loss: 0.0219 - acc: 0.9945 - val_loss: 0.0835 - val_acc: 0.9711
Epoch 5/21
125/124 [==============================] - 294s 2s/step - loss: 0.0249 - acc: 0.9925 - val_loss: 0.0757 - val_acc: 0.9711
Epoch 6/21
125/124 [==============================] - 294s 2s/step - loss: 0.0235 - acc: 0.9927 - val_loss: 0.0752 - val_acc: 0.9750
Epoch 7/21
125/124 [==============================] - 295s 2s/step - loss: 0.0241 - acc: 0.9925 - val_loss: 0.0782 - val_acc: 0.9697
Epoch 8/21
125/124 [==============================] - 294s 2s/step - 

In [12]:
# ------ TESTING ------
x_test = []
df_test = pd.read_csv('/input/plant_seedlings_classification/sample_submission.csv')

for f, species in tqdm(df_test.values, miniters=100):
    img = cv2.imread('/input/plant_seedlings_classification/test/{}'.format(f))
    x_test.append(cv2.resize(img, (dim, dim)))

x_test = np.array(x_test, np.float32)
print(x_test.shape)

x_test = x_test /255.0

if os.path.isfile(weights):
    model.load_weights(weights)

p_test = model.predict(x_test, verbose = 1)

preds = []
for i in range(len(p_test)):
    pos = np.argmax(p_test[i])
    preds.append(list(label_map.keys())[list(label_map.values()).index(pos)])

df_test['species'] = preds
df_test.to_csv('/output/submission.csv', index=False)

100%|██████████| 794/794 [00:05<00:00, 145.59it/s]


(794, 256, 256, 3)
